In [ ]:
!pip install -U "tensorflow>=2.5"

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
!tar -xvf /content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/datasets/annotations.tar.gz

In [ ]:
!tar -xvf /content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/datasets/images.tar.gz

In [ ]:
!cp "/content/models/research/object_detection/data/pet_label_map.pbtxt" \
"/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/annotations"

In [ ]:
!mv "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/annotations/pet_label_map.pbtxt" \
"/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/annotations/label_map.pbtxt"

In [ ]:
!python /content/models/research/object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path=/content/models/research/object_detection/data/pet_label_map.pbtxt \
    --data_dir=/content/train_images \
    --output_dir=/content/drive/MyDrive/Tensorflow_OD_API/efficientDet_D1_640x640/workspace/training_demo/annotations/records

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz

In [ ]:
!mkdir "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1"

In [ ]:
!cp "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/pipeline.config" \
"/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1" \
--pipeline_config_path="/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1/pipeline.config"

**모델 저장하기**

In [ ]:
!mkdir "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1"

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1/pipeline.config" \
--trained_checkpoint_dir "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1" \
--output_directory "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1" \
--pipeline_config_path "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1/pipeline.config" \
--checkpoint_dir="/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1"

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/models/myft_efficientDet_d1"

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
%matplotlib inline

In [ ]:
PATH_TO_MODEL_DIR = "/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/exported-models/saved_efficientDet_D1"
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print(PATH_TO_SAVED_MODEL)

In [ ]:
import tensorflow as tf

In [ ]:
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap("/content/drive/MyDrive/Tensorflow_OD_API/EfficientDet_D1_640x640/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)

In [ ]:
imageDir2 = "/content/cat"
IMAGE_PATHS = [imageDir2 + "/" + f for f in os.listdir(imageDir2) if f[-4:] in ['.jpg','.jpeg','.png']]
IMAGE_PATHS = IMAGE_PATHS[0:9]

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

In [ ]:
for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure(figsize=(12,12))
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()